In [82]:
import pandas as pd
import numpy as np
import string
import itertools
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import yake
print("done")

done


In [83]:
import csv

input_file = 'clustering_data.csv'   # your original CSV
output_file = 'clustering_data_clean.csv'  # cleaned CSV

with open(input_file, 'r', encoding='utf-8') as infile, \
     open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)
    
    # Read header
    try:
        header = next(reader)
    except StopIteration:
        print("CSV is empty!")
        exit()
    
    if len(header) != 4:
        print(f"Warning: Header has {len(header)} columns, expected 4")
    
    writer.writerow(header)  # write header to cleaned file
    
    for row in reader:
        # Skip empty rows
        if not row or all(cell.strip() == '' for cell in row):
            continue
        
        # Only keep rows with exactly 4 columns
        if len(row) != 4:
            print(f"Skipping malformed row: {row}")
            continue
        
        # Write row safely (wrap text in quotes)
        writer.writerow(row)

print(f"Cleaned CSV saved as: {output_file}")


Skipping malformed row: ['5', 'The water supply is highly irregular', ' especially in the mornings.', 'Maintenance', '2025-08-17']
Skipping malformed row: ['12', "Doors don't lock properly", ' posing security risks.', 'Hostel', '2025-08-28']
Skipping malformed row: ['21', 'Unhygienic washrooms', ' cleaning staff rarely attends.', 'Hostel', '2025-08-03']
Skipping malformed row: ['26', 'The water supply is highly irregular', ' especially in the mornings.', 'Maintenance', '2025-08-10']
Skipping malformed row: ['31', "Doors don't lock properly", ' posing security risks.', 'Hostel', '2025-07-16']
Skipping malformed row: ['42', 'The water supply is highly irregular', ' especially in the mornings.', 'Maintenance', '2025-08-05']
Skipping malformed row: ['47', "Doors don't lock properly", ' posing security risks.', 'Hostel', '2025-08-21']
Skipping malformed row: ['56', 'The water supply is highly irregular', ' especially in the mornings.', 'Maintenance', '2025-08-07']
Skipping malformed row: ['

In [84]:
import os
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer


In [85]:
df = pd.read_csv('clustering_data_clean.csv')
df.head()


id                                     complaint_text     category  \
0   1  Room lights malfunction every week and nobody ...  Maintenance   
1   2                                  Mess food is bad.       Hostel   
2   3  Lectures are rescheduled last minute without n...      Faculty   
3   4        Mess staff misbehaves and ignores requests.       Hostel   
4   6  Professors are rarely available for doubt clea...      Faculty   

    timestamp  
0  2025-08-06  
1  2025-07-27  
2  2025-09-02  
3  2025-07-22  
4  2025-07-12

In [86]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['complaint_text'].apply(preprocess)
df.head()


id                                     complaint_text     category  \
0   1  Room lights malfunction every week and nobody ...  Maintenance   
1   2                                  Mess food is bad.       Hostel   
2   3  Lectures are rescheduled last minute without n...      Faculty   
3   4        Mess staff misbehaves and ignores requests.       Hostel   
4   6  Professors are rarely available for doubt clea...      Faculty   

    timestamp                                         clean_text  
0  2025-08-06   room light malfunction every week nobody attends  
1  2025-07-27                                      mess food bad  
2  2025-09-02  lecture rescheduled last minute without notifi...  
3  2025-07-22              mess staff misbehaves ignores request  
4  2025-07-12         professor rarely available doubt clearance

In [87]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [88]:
import pandas as pd
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [89]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    # Lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize
    tokens = text.split()
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)


In [90]:
df['clean_text'] = df['complaint_text'].apply(preprocess)

# Print first 5 rows to check
print(df[['complaint_text', 'clean_text']].head())


                                      complaint_text  \
0  Room lights malfunction every week and nobody ...   
1                                  Mess food is bad.   
2  Lectures are rescheduled last minute without n...   
3        Mess staff misbehaves and ignores requests.   
4  Professors are rarely available for doubt clea...   

                                          clean_text  
0   room light malfunction every week nobody attends  
1                                      mess food bad  
2  lecture rescheduled last minute without notifi...  
3              mess staff misbehaves ignores request  
4         professor rarely available doubt clearance  


In [91]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
print(" Model loaded successfully!")


 Model loaded successfully!


In [92]:
embeddings = model.encode(df['clean_text'].tolist(), convert_to_numpy=True)

os.makedirs('embeddings', exist_ok=True)
np.save('embeddings/embeddings.npy', embeddings)

print(f"✅ Embeddings generated! Shape: {embeddings.shape}")

✅ Embeddings generated! Shape: (418, 384)


In [93]:
# Cell 5: Compute Similarity Matrix
from sklearn.metrics.pairwise import cosine_similarity

sim_matrix = cosine_similarity(embeddings)
print(sim_matrix.shape)


(418, 418)


In [94]:
# Cell 6: Define Custom Similarity + Final Score
def jaccard_similarity(text1, text2):
    """
    Compute Jaccard similarity between two texts based on word overlap.
    """
    set1 = set(text1.split())
    set2 = set(text2.split())
    return len(set1 & set2) / len(set1 | set2)

def final_score(i, j):
    """
    Compute the final similarity score between complaint i and complaint j.
    Combines:
    - Cosine similarity of embeddings
    - Jaccard keyword overlap
    - Category match (if available)
    """
    cosine_sim = sim_matrix[i][j]
    keyword_overlap = jaccard_similarity(df['clean_text'][i], df['clean_text'][j])
    category_match = 1 if df['category'][i] == df['category'][j] else 0
    return (0.7 * cosine_sim) + (0.2 * keyword_overlap) + (0.1 * category_match)


In [95]:
# Cell 6: Define Dynamic Threshold Function
import numpy as np

def dynamic_threshold(i, base=0.25, scale=0.2):
    """
    Compute a dynamic threshold for each complaint i based on
    the mean and standard deviation of its similarity distribution.
    This makes the threshold adaptive per complaint.
    """
    sims = sim_matrix[i]  # Ensure sim_matrix is already defined
    mean_sim = np.mean(sims)
    std_sim = np.std(sims)
    return base + scale * std_sim + 0.1 * mean_sim  # more context-sensitive threshold


In [96]:
# Cell 7: Clustering Using Dynamic Threshold
visited = set()
clusters = []

for i in range(len(df)):
    if i in visited:
        continue
    
    current_cluster = [i]
    visited.add(i)
    thr = dynamic_threshold(i)
    
    for j in range(len(df)):
        if j in visited:
            continue
        if final_score(i, j) > thr:
            current_cluster.append(j)
            visited.add(j)
    
    clusters.append(current_cluster)

# Show number of clusters formed
print(f"Clusters formed: {len(clusters)}")


Clusters formed: 23


In [97]:
#Check Cluster Sizes
cluster_sizes = [len(c) for c in clusters]
print("Number of clusters:", len(clusters))
print("Average cluster size:", sum(cluster_sizes)/len(cluster_sizes))
print("Clusters with size 1:", sum(1 for c in clusters if len(c) == 1))


Number of clusters: 23
Average cluster size: 18.17391304347826
Clusters with size 1: 4


In [98]:
#Calculating confidense score for clusters in detail clustering
import itertools
import numpy as np
def cluster_confidence(cluster, sim_matrix):
    if len(cluster) < 2:
        return 1.0  # single complaint = perfect confidence
    pairwise_sims = [sim_matrix[i][j] for i, j in itertools.combinations(cluster, 2)]
    return np.mean(pairwise_sims)

# Calculate confidence for all clusters
cluster_confidences = [cluster_confidence(c, sim_matrix) for c in clusters]

for i, (c, conf) in enumerate(zip(clusters, cluster_confidences)):
    print(f"Cluster {i+1} | Size: {len(c)} | Confidence: {conf:.2f}")


Cluster 1 | Size: 64 | Confidence: 0.39
Cluster 2 | Size: 70 | Confidence: 0.46
Cluster 3 | Size: 79 | Confidence: 0.42
Cluster 4 | Size: 31 | Confidence: 0.46
Cluster 5 | Size: 46 | Confidence: 0.43
Cluster 6 | Size: 35 | Confidence: 0.41
Cluster 7 | Size: 31 | Confidence: 0.52
Cluster 8 | Size: 2 | Confidence: 0.36
Cluster 9 | Size: 18 | Confidence: 0.66
Cluster 10 | Size: 6 | Confidence: 0.44
Cluster 11 | Size: 4 | Confidence: 0.38
Cluster 12 | Size: 6 | Confidence: 1.00
Cluster 13 | Size: 3 | Confidence: 0.33
Cluster 14 | Size: 3 | Confidence: 0.35
Cluster 15 | Size: 6 | Confidence: 0.34
Cluster 16 | Size: 3 | Confidence: 0.40
Cluster 17 | Size: 2 | Confidence: 0.31
Cluster 18 | Size: 1 | Confidence: 1.00
Cluster 19 | Size: 3 | Confidence: 0.25
Cluster 20 | Size: 1 | Confidence: 1.00
Cluster 21 | Size: 1 | Confidence: 1.00
Cluster 22 | Size: 2 | Confidence: 0.49
Cluster 23 | Size: 1 | Confidence: 1.00


In [99]:
def representative_complaint(cluster, df):
    # Option 1: Pick the longest complaint
    return max(cluster, key=lambda idx: len(df['complaint_text'][idx]))

# Example: pick representative for first cluster
rep_idx = representative_complaint(clusters[0], df)
print("Representative Complaint:", df['complaint_text'][rep_idx])


Representative Complaint: The classroom fans are broken and have not been fixed. Several maintenance requests were filed with no resolution. The issue has inconvenienced students for weeks and some have missed assessments as a direct result.


In [100]:
for i, cluster in enumerate(clusters, start=1):
    print(f"\n🔹 Cluster {i} | Size: {len(cluster)} | Confidence: {cluster_confidences[i-1]:.2f}")
    for idx in cluster:
        print(f"   - {df['complaint_text'][idx]} ({df['category'][idx]})")



🔹 Cluster 1 | Size: 64 | Confidence: 0.39
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - The classroom fans are broken and have not been fixed. Several maintenance requests were filed with no resolution. The issue has inconvenienced students for weeks and some have missed assessments as a direct result. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody attends. (Maintenance)
   - Room lights malfunction every week and nobody atten

In [101]:
cluster_texts = []

for c in clusters:
    # Join all complaints in the cluster into one string
    cluster_text = " ".join(df['complaint_text'][i] for i in c)
    cluster_texts.append(cluster_text)


In [102]:
cluster_summaries = []

for text in cluster_texts:
    # Split by period
    sentences = text.split('.')
    # Take first 2–3 sentences for short summary
    short_summary = '. '.join(sentences[:3]).strip()
    cluster_summaries.append(short_summary)


In [103]:
import yake

kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)

final_cluster_summaries = []

for idx, text in enumerate(cluster_texts):
    sentences = text.split('.')
    short_summary = '. '.join(sentences[:3]).strip()
    
    # Extract keywords from cleaned text
    cluster_clean_text = " ".join(df['clean_text'][i] for i in clusters[idx])
    keywords = [kw for kw, score in kw_extractor.extract_keywords(cluster_clean_text)]
    
    final_summary = f"{short_summary} | Keywords: {', '.join(keywords)}"
    final_cluster_summaries.append(final_summary)


In [104]:
print(f"Number of clusters: {len(clusters)}")
print(f"First 5 clusters: {clusters[:5]}")


Number of clusters: 23
First 5 clusters: [[0, 17, 26, 44, 53, 65, 79, 89, 99, 116, 125, 138, 145, 151, 153, 157, 164, 168, 170, 177, 185, 188, 190, 192, 194, 198, 201, 224, 228, 232, 234, 237, 238, 242, 246, 250, 253, 255, 258, 263, 267, 271, 275, 279, 284, 291, 297, 313, 321, 329, 332, 342, 346, 350, 358, 362, 374, 386, 389, 392, 400, 404, 412, 416], [1, 3, 11, 15, 20, 27, 30, 35, 40, 43, 50, 54, 60, 63, 69, 72, 80, 82, 88, 94, 98, 103, 107, 110, 115, 121, 123, 128, 134, 155, 162, 179, 182, 183, 186, 199, 207, 212, 214, 220, 222, 230, 236, 240, 244, 248, 257, 261, 273, 277, 289, 299, 303, 307, 311, 315, 319, 323, 328, 333, 336, 340, 348, 352, 360, 364, 368, 376, 391, 394], [2, 7, 18, 22, 32, 36, 41, 45, 49, 55, 59, 66, 70, 75, 81, 87, 90, 93, 100, 106, 109, 118, 120, 129, 132, 133, 140, 146, 147, 154, 161, 165, 169, 171, 173, 175, 178, 181, 189, 193, 195, 202, 219, 225, 229, 233, 235, 243, 247, 256, 260, 264, 268, 272, 282, 292, 294, 298, 306, 314, 318, 326, 327, 330, 339, 347, 351, 3

In [105]:
print(df.head())
print(df.columns)

   id                                     complaint_text     category  \
0   1  Room lights malfunction every week and nobody ...  Maintenance   
1   2                                  Mess food is bad.       Hostel   
2   3  Lectures are rescheduled last minute without n...      Faculty   
3   4        Mess staff misbehaves and ignores requests.       Hostel   
4   6  Professors are rarely available for doubt clea...      Faculty   

    timestamp                                         clean_text  
0  2025-08-06   room light malfunction every week nobody attends  
1  2025-07-27                                      mess food bad  
2  2025-09-02  lecture rescheduled last minute without notifi...  
3  2025-07-22              mess staff misbehaves ignores request  
4  2025-07-12         professor rarely available doubt clearance  
Index(['id', 'complaint_text', 'category', 'timestamp', 'clean_text'], dtype='object')


In [106]:
import string
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['complaint_text'].apply(preprocess)


In [107]:
import yake

kw_extractor = yake.KeywordExtractor(top=5, stopwords=None)

final_cluster_summaries = []

for idx, c in enumerate(clusters):
    if not c:
        final_cluster_summaries.append("Empty cluster")
        continue
    
    # Combine all complaints
    cluster_text = " ".join(df['complaint_text'][i] for i in c)
    
    # Take first 2 sentences (simple split)
    sentences = cluster_text.split('.')
    short_summary = '. '.join(sentences[:2]).strip()
    
    # Keywords from cleaned text
    cluster_clean_text = " ".join(df['clean_text'][i] for i in c)
    keywords = [kw for kw, score in kw_extractor.extract_keywords(cluster_clean_text)]
    
    summary = f"{short_summary} | Keywords: {', '.join(keywords)}"
    final_cluster_summaries.append(summary)
for i, s in enumerate(final_cluster_summaries):
    print(f"Cluster {i} summary: {s}\n")


Cluster 0 summary: Room lights malfunction every week and nobody attends.  The classroom fans are broken and have not been fixed | Keywords: room light malfunction, attends room light, heating system malfunction, hostel common room, study room malfunction

Cluster 1 summary: Mess food is bad.  Mess staff misbehaves and ignores requests | Keywords: mess food bad, food bad mess, request mess food, bad mess staff, mess staff misbehaves

Cluster 2 summary: Lectures are rescheduled last minute without notifications.  Delay in releasing project grades is affecting internships | Keywords: releasing project grade, delay releasing project, project grade affecting, grade affecting internship, notification delay releasing

Cluster 3 summary: Professors are rarely available for doubt clearance.  Professors are rarely available for doubt clearance | Keywords: doubt clearance professor, clearance professor rarely, doubt clearance lack, doubt clearance, professor rarely

Cluster 4 summary: Broken cha